TF

In [1]:
import seismic_ml_kernel

In [2]:
#train (train_data_dir, validation_data_dir, image_size)
#test (validation_data_dir + 'noise/', image_size)

In [3]:
from keras.datasets import mnist # subroutines for fetching the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data() # fetch MNIST data

#mn (X_train, y_train, X_test, y_test)

Using TensorFlow backend.


In [4]:
from keras.datasets import cifar10 # subroutines for fetching the CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data() # fetch CIFAR-10 data

#ci (X_train, y_train, X_test, y_test)

In [3]:
def seis_dataset ():
    import numpy as np
    import seismic_handler
    from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
    
    noisers = [None, seismic_handler.FreqNoiser()]
    
    images_dir = 'images/'

    import os
    if os.path.exists(images_dir):
        import shutil
        shutil.rmtree(images_dir)

    if not os.path.exists(images_dir):
        os.makedirs(images_dir)

    train_data_dir = images_dir + 'train/'
    validation_data_dir = images_dir + 'validation/'

    if not os.path.exists(train_data_dir):
        os.makedirs(train_data_dir)

    if not os.path.exists(validation_data_dir):
        os.makedirs(validation_data_dir)
        
    for i in range(len(noisers)):
        a = train_data_dir + str(i)
        if not os.path.exists(a):
            os.makedirs(a)
    
        a = validation_data_dir + str(i)
        if not os.path.exists(a):
            os.makedirs(a)
    
    handlers = [seismic_handler.SeismicPrestack('data/prestack.sgy', noiser) for noiser in noisers]
    gather_keys = [handler.getHeaderVals ('original_field_record_number') for handler in handlers]
        
    X_train = []
    y_train = []
    X_test = [] 
    y_test = []
    
    counter = 0
    nhandlers = len (handlers)
           
    for key in gather_keys[0]:
        partss = [handler.readGatherParts (key, 50, 0.5) for handler in handlers]

        nparts = len(partss[0])

        sh = partss[0][0].shape
        for i in range(nparts):                
            category = i % nhandlers
            data = partss[category][i]

            if data.shape != sh:
                throw ('wrong shapes')

            image = seismic_handler.convert_to_image (data.T)
            data = img_to_array (image)

            if i < nparts*0.8:
                X_train.append(data)
                y_train.append (category)
                png_name = train_data_dir
            else:
                X_test.append(data)
                y_test.append(category)   
                png_name = validation_data_dir
                
            png_name += str (category) + '/'    
            png_name += str(counter) + '.png'
            image.save (png_name)
                
            counter = counter+1
    
    y_test = np.array(y_test)
    y_train = np.array(y_train)
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    
    
    if len(X_train) != len(y_train):
        print ('len(X_train) != len(y_train)')
        return
    if len(X_test) != len(y_test):
        print ('len(X_test) != len(y_test)')
        return

    return (X_train, y_train), (X_test, y_test)

In [4]:
import seismic_ml_kernel

(X_train, y_train), (X_test, y_test) = seis_dataset ()

seismic_ml_kernel.ci (X_train, y_train, X_test, y_test)

C:\Users\kostyad\Documents\Jupyter\seismic_fingerprint\seismic_ml_kernel.py:78: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
  conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(inp)
C:\Users\kostyad\Documents\Jupyter\seismic_fingerprint\seismic_ml_kernel.py:79: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
  conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_1)
C:\Users\kostyad\Documents\Jupyter\seismic_fingerprint\seismic_ml_kernel.py:83: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
  conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
C:\Users\kostyad\Documents\Jupyter\seismic_fingerprint\seismic_ml_kernel.py:84: Us

Train on 11686 samples, validate on 1299 samples
Epoch 1/200
 1216/11686 [==>...........................] - ETA: 373s - loss: 0.7046 - acc: 0.4860

KeyboardInterrupt: 